In [1]:
from Bio import SeqIO
from Bio import Entrez
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
import numpy as np
import pandas as pd

## Table Explanations
#### Genes Table: gene id, organism name, sequence, enzyme, description
#### Enzymes Table: name, function, enzyme number
#### Pathway Table: enzyme names, pathway 
#### Associative Tables (connectors): genes and enzymes, pathways and enzymes 

#### Questions
##### 1. The enzyme to pathway is many to many because many pathways exist using the same enzymes, and many enzymes (non-unique) are used in each pathway. This can further be generalized to say that multiple rows in first table can link to multiple rows in the second table and vice versa. 

##### 2. An order can be used by indices in a table to describe a pathway and ordering that would not be apparent in an ordinary linkage. The enzyme can hold many numerical attributes for its order in various pathways. 

##### 3. This relationship can best be represented by a 2d array, because of Beadle and Tatums experimentation we understand that gene-to-protein is a one-to-one relatioship, pointing from gene to enzyme. 

### Extract_gene fetches the nucleotide sequence for an enzyme of a particular organism from the Entrez database

In [2]:
def extract_gene(enzyme, org):
    Entrez.email = 'sylviaillouz@berkeley.edu'
    handle = Entrez.esearch(db ='nucleotide', term = enzyme + ' AND ' + org + '[porgn]', sort ='relevance', idtype ='acc')

    i = Entrez.read(handle)['IdList'][0]
    handle = Entrez.efetch(db ='nucleotide', id = i, rettype = 'gb', retmode = 'text', retmax = 1)
    record = SeqIO.read(handle, 'genbank')
    return record
# an example below
extract_gene('enolase', '"Homo sapiens"')

SeqRecord(seq=Seq('ATCGGATCTGAGCGAACGGAACGGGTGCGGGTGTTCAAGATGTCCATTCTCAAG...AAA', IUPACAmbiguousDNA()), id='X66610.1', name='X66610', description='H.sapiens mRNA for enolase', dbxrefs=[])

### Chose Enzymes for 3 pathways in common with all three organisms E.Colo, Drysophila, and Homo Sapiens

In [3]:

# GLYCOLYSIS: phosphoglucomutase, glucose-6-phosphate isomerase,
#             aldose 1-epimerase, 6-phosphofructokinase

glycolysis_enzymes = ['phosphoglucomutase', 'glucose-6-phosphate isomerase',
                     'aldose 1-epimerase', '6-phosphofructokinase']

# PENTOSE PHOSPHATE: enolase, phosphoglycerate kinase, 
#                    glyceraldehyde 3-phosphate dehydrogenase,
#                    triosephosphate isomerase

pentosephosphate_enzymes = ['enolase', 'phosphoglycerate kinase',
                           'glyceraldehyde 3-phosphate dehydrogenase',
                           'triosephosphate isomerase']

# CITRATE: dihydrolipoamide dehydrogenase, alcohol dehydrogenase,
#          acetyl-CoA synthetase, pyruvate kinase

citrate_enzymes = ['dihydrolipoamide dehydrogenase', 'alcohol dehydrogenase',
                  'acetyl-CoA synthetase', 'pyruvate kinase']
pathways = [glycolysis_enzymes, pentosephosphate_enzymes, citrate_enzymes]

## 1. Enzyme Table

In [4]:
c.execute("DROP TABLE enzyme_table;")
c.execute("CREATE TABLE enzyme_table (enzyme_name TEXT, function TEXT, enzyme_num TEXT);")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('phosphoglucomutase', 'isomerase', '5.4.22');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('glucose-6-phosphate isomerase', 'isomerase', '5.3.1.9');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('aldose 1-epimerase', 'isomerase', '5.1.3.3');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('6-phosphofructokinase', 'transferase', '2.7.1.11');")

c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('enolase', 'lyase', '4.2.1.11');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('phosphoglycerate kinase', 'isomerase', '5.3.1.9');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('glyceraldehyde 3-phosphate dehydrogenase', 'oxidoreductase', '1.2.1.12');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('triosephosphate isomerase', 'isomerase', '5.3.1.1');")

c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('dihydrolipoamide dehydrogenase', 'oxidoreductase', '1.8.1.4');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('alcohol dehydrogenase', 'oxidoreductase', '1.1.1.1');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('acetyl-CoA synthase', 'ligase', '6.2.1.1');")
c.execute("INSERT INTO enzyme_table (enzyme_name, function, enzyme_num) VALUES ('pyruvate kinase', 'transferase', '2.7.1.40');")
conn.commit()
c.execute("SELECT * FROM enzyme_table;")
for elem in c.fetchall():
    print(elem)

('phosphoglucomutase', 'isomerase', '5.4.22')
('glucose-6-phosphate isomerase', 'isomerase', '5.3.1.9')
('aldose 1-epimerase', 'isomerase', '5.1.3.3')
('6-phosphofructokinase', 'transferase', '2.7.1.11')
('enolase', 'lyase', '4.2.1.11')
('phosphoglycerate kinase', 'isomerase', '5.3.1.9')
('glyceraldehyde 3-phosphate dehydrogenase', 'oxidoreductase', '1.2.1.12')
('triosephosphate isomerase', 'isomerase', '5.3.1.1')
('dihydrolipoamide dehydrogenase', 'oxidoreductase', '1.8.1.4')
('alcohol dehydrogenase', 'oxidoreductase', '1.1.1.1')
('acetyl-CoA synthase', 'ligase', '6.2.1.1')
('pyruvate kinase', 'transferase', '2.7.1.40')


## 2. Genes Table

In [8]:
org = []
gene_name = []
description = []
seq = []
product = []
data = {'gene_name': gene_name,
       'description': description,
        'org': org,
       'seq': seq,
       'product': product}
# first creating numpy arrays of information for homo sapiens and drosophila only
# prokaryotes are slightly different... see below
for organism in ['"Homo sapiens"', '"Drosophila"']:
    #iterate through enzymes and populate arrays
    for enzyme in glycolysis_enzymes:
        print(enzyme)
        gene = extract_gene(enzyme, organism)
        seq.append(str(gene.seq))
        org.append(str(gene.features[0].qualifiers['organism']))
        gene_name.append(gene.name)
        description.append(gene.description)
        product.append(str(enzyme))
    print(data)
    for enzyme in pentosephosphate_enzymes:
        print(enzyme)
        gene = extract_gene(enzyme, organism)
        seq.append(str(gene.seq))
        org.append(str(gene.features[0].qualifiers['organism']))
        gene_name.append(gene.name)
        description.append(gene.description)
        product.append(str(enzyme))
    for enzyme in citrate_enzymes:
        gene = extract_gene(enzyme, organism)
        seq.append(str(gene.seq))
        org.append(str(gene.features[0].qualifiers['organism']))
        gene_name.append(gene.name)
        description.append(gene.description)
        product.append(str(enzyme))


        
        

phosphoglucomutase
glucose-6-phosphate isomerase
aldose 1-epimerase
6-phosphofructokinase
{'gene_name': ['HUMPGM1A', 'KY379509', 'BC019263', 'HUMPFK6PHO'], 'description': ['Human phosphoglucomutase 1 (PGM1) mRNA, complete cds', 'Homo sapiens cell line HeLa glucose-6-phosphate isomerase mRNA, complete cds', 'Homo sapiens galactose mutarotase (aldose 1-epimerase), mRNA (cDNA clone MGC:3215 IMAGE:3502667), complete cds', "Homo sapiens 6-phosphofructo-1-kinase (PFK) DNA, 5' end"], 'org': ["['Homo sapiens']", "['Homo sapiens']", "['Homo sapiens']", "['Homo sapiens']"], 'seq': ['GGGCCGGCCGCCCCTCCGCCAGCCAAGTCCGCCGCTCTGACCCCCGGCAGCAAGTCGCCACCATGGTGAAGATCGTGACAGTTAAGACCCAGGCGTACCAGGACCAGAAGCCGGGCACGAGCGGGCTGCGGAAGCGGGTGAAGGTGTTCCAGAGCAGCGCCAACTACGCGGAGAACTTCATCCAGAGTATCATCTCCACCGTGGAGCCGGCGCAGCGGCAGGAGGCCACGCTGGTGGTGGGCGGGGACGGCCGGTTCTACATGAAGGAGGCCATCCAGCTCATCGCTCGCATCGCTGCCGCCAACGGGATCGGTCGCTTGGTTATCGGACAGAATGGAATCCTCTCCACCCCTGCTGTATCCTGCATCATTAGAAAAATCAAAGCCATTGGTGGGATCATTCTGACAGCCAGTCACAACC

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



phosphoglycerate kinase
glyceraldehyde 3-phosphate dehydrogenase
triosephosphate isomerase


In [9]:
df2 = pd.DataFrame.from_dict(data)
df2

,gene_name,description,org,seq,product
0,HUMPGM1A,"Human phosphoglucomutase 1 (PGM1) mRNA, comple...",['Homo sapiens'],GGGCCGGCCGCCCCTCCGCCAGCCAAGTCCGCCGCTCTGACCCCCG...,phosphoglucomutase
1,KY379509,Homo sapiens cell line HeLa glucose-6-phosphat...,['Homo sapiens'],ATGGCCGCTCTCACCCGGGACCCCCAGTTCCAGAAGCTGCAGCAAT...,glucose-6-phosphate isomerase
2,BC019263,Homo sapiens galactose mutarotase (aldose 1-ep...,['Homo sapiens'],CCTCTAGCTTAGCGAGCGCTGGAGTTTGAAGAGCGGGCAGTGGCTG...,aldose 1-epimerase
3,HUMPFK6PHO,Homo sapiens 6-phosphofructo-1-kinase (PFK) DN...,['Homo sapiens'],CTCATTCTATTCAGTCAACTCTCTTTTCCCTGACCTTAGTTTATTC...,6-phosphofructokinase
4,X66610,H.sapiens mRNA for enolase,['Homo sapiens'],ATCGGATCTGAGCGAACGGAACGGGTGCGGGTGTTCAAGATGTCCA...,enolase
5,AH002938,Homo sapiens phosphoglycerate kinase (PGK1) ge...,['Homo sapiens'],TCCACGGGGTTGGGGTTGCGCCTTTTCCAAGGCAGCCCTGGGTTTG...,phosphoglycerate kinase
6,X01677,Human liver mRNA for glyceraldehyde-3-phosphat...,['Homo sapiens'],GCTCGGCTGGCGACGCAAAAGAAGATGCGGCTGACTGTCGAGCCAC...,glyceraldehyde 3-phosphate dehydrogenase
7,X69723,H.sapiens TPI1 gene for triosephosphate isomerase,['Homo sapiens'],CTGCAGTTCCTGCCAGGCCTTGCCAGCCGGGGCGAGGGTTGGGATG...,triosephosphate isomerase
8,AH003583,Homo sapiens haplotype lambda Charon 4A dihydr...,['Homo sapiens'],ATGCTGTAACACAGCGAAGGTTTGCAGCTTCACTCCTGAAGCCAGT...,dihydrolipoamide dehydrogenase
9,HUMALDE,Human class IV sigma-1 alcohol dehydrogenase m...,['Homo sapiens'],ATGGGCACTGCTGGAAAAGTTATTAAGTGCAAAGCAGCTGTGCTTT...,alcohol dehydrogenase


In [10]:

# need new handle for e.coli because genome is now on one circular plasmid,
# can extract all genes (unlike for eukaryotic cells above)
# code below taken from lab4 files
# took code samples from https://biopython.org/wiki/Intergenic_regions
Entrez.email = 'sylviaillouz@berkeley.edu'
handle = Entrez.esearch(db = 'nucleotide', term = 'MG1655 AND escherichia coli[orgn]', sort = 'relevance', idtype = 'acc')
i = Entrez.read(handle)['IdList'][7]
handle = Entrez.efetch(db = 'nucleotide', id = i, rettype = 'gb', retmode = 'text', retmax = 1)
gene_map = {}
for record in SeqIO.parse(handle, "genbank"):
    ec_genome = record.seq
    for elem in record.features:
        if elem.type == "CDS":
            element = elem.qualifiers["product"][0]
            gene_map[element] = elem
list1 = list(gene_map.items())
print(list1[0])
for pathway in pathways:
    for enzyme in pathway:
        genes_of_interest = {}
        if enzyme in gene_map.keys():
            gene = gene_map[enzyme]
            product.append(str(enzyme))
            gene_name.append(gene.qualifiers['gene'][0])
            description.append(gene.qualifiers['gene_synonym'][0])
            org.append('EColi')
            seq.append(ec_genome[gene.location.start:gene.location.end])
        


('thr operon leader peptide', SeqFeature(FeatureLocation(ExactPosition(189), ExactPosition(255), strand=1), type='CDS'))


In [13]:
df3 = pd.DataFrame(data)
df3

,gene_name,description,org,seq,product
0,HUMPGM1A,"Human phosphoglucomutase 1 (PGM1) mRNA, comple...",['Homo sapiens'],GGGCCGGCCGCCCCTCCGCCAGCCAAGTCCGCCGCTCTGACCCCCG...,phosphoglucomutase
1,KY379509,Homo sapiens cell line HeLa glucose-6-phosphat...,['Homo sapiens'],ATGGCCGCTCTCACCCGGGACCCCCAGTTCCAGAAGCTGCAGCAAT...,glucose-6-phosphate isomerase
2,BC019263,Homo sapiens galactose mutarotase (aldose 1-ep...,['Homo sapiens'],CCTCTAGCTTAGCGAGCGCTGGAGTTTGAAGAGCGGGCAGTGGCTG...,aldose 1-epimerase
3,HUMPFK6PHO,Homo sapiens 6-phosphofructo-1-kinase (PFK) DN...,['Homo sapiens'],CTCATTCTATTCAGTCAACTCTCTTTTCCCTGACCTTAGTTTATTC...,6-phosphofructokinase
4,X66610,H.sapiens mRNA for enolase,['Homo sapiens'],ATCGGATCTGAGCGAACGGAACGGGTGCGGGTGTTCAAGATGTCCA...,enolase
5,AH002938,Homo sapiens phosphoglycerate kinase (PGK1) ge...,['Homo sapiens'],TCCACGGGGTTGGGGTTGCGCCTTTTCCAAGGCAGCCCTGGGTTTG...,phosphoglycerate kinase
6,X01677,Human liver mRNA for glyceraldehyde-3-phosphat...,['Homo sapiens'],GCTCGGCTGGCGACGCAAAAGAAGATGCGGCTGACTGTCGAGCCAC...,glyceraldehyde 3-phosphate dehydrogenase
7,X69723,H.sapiens TPI1 gene for triosephosphate isomerase,['Homo sapiens'],CTGCAGTTCCTGCCAGGCCTTGCCAGCCGGGGCGAGGGTTGGGATG...,triosephosphate isomerase
8,AH003583,Homo sapiens haplotype lambda Charon 4A dihydr...,['Homo sapiens'],ATGCTGTAACACAGCGAAGGTTTGCAGCTTCACTCCTGAAGCCAGT...,dihydrolipoamide dehydrogenase
9,HUMALDE,Human class IV sigma-1 alcohol dehydrogenase m...,['Homo sapiens'],ATGGGCACTGCTGGAAAAGTTATTAAGTGCAAAGCAGCTGTGCTTT...,alcohol dehydrogenase


## 3. Pathway Table

In [14]:
c.execute("DROP TABLE pathways_table;")
c.execute("CREATE TABLE pathways_table (pathway_name TEXT, description TEXT);")
c.execute("INSERT INTO pathways_table (pathway_name, description) VALUES ('glycolysis', 'metabolic pathway that converts glucose C6H12O6 into pyruvate CH3COCOO− + H+ The free energy released in this process is used to form the high-energy molecules ATP (adenosine triphosphate) and NADH (reduced nicotinamide adenine dinucleotide).');")
c.execute("INSERT INTO pathways_table (pathway_name, description) VALUES ('pentophosphate', 'a metabolic pathway parallel to glycolysis. It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate, the last one a precursor for the synthesis of nucleotides.');")
c.execute("INSERT INTO pathways_table (pathway_name, description) VALUES ('citrate', 'starts with acetyl-CoA, the activated form of acetate, derived from glycolysis and pyruvate oxidation for carbohydrates and from beta oxidation of fatty acids.');")

c.execute("SELECT * FROM pathways_table;")
conn.commit()
for elem in c.fetchall():
    print(elem)


('glycolysis', 'metabolic pathway that converts glucose C6H12O6 into pyruvate CH3COCOO− + H+ The free energy released in this process is used to form the high-energy molecules ATP (adenosine triphosphate) and NADH (reduced nicotinamide adenine dinucleotide).')
('pentophosphate', 'a metabolic pathway parallel to glycolysis. It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate, the last one a precursor for the synthesis of nucleotides.')
('citrate', 'starts with acetyl-CoA, the activated form of acetate, derived from glycolysis and pyruvate oxidation for carbohydrates and from beta oxidation of fatty acids.')


## 4. Associated Tables

In [15]:
# ENZYME TO PATHWAY TABLE
c.execute("DROP TABLE enzyme_to_pathway;")
c.execute("CREATE TABLE enzyme_to_pathway (enzyme_name TEXT, pathway_name TEXT);")
for enzyme in glycolysis_enzymes:
    c.execute("INSERT INTO enzyme_to_pathway (enzyme_name, pathway_name) VALUES (?,?);", (enzyme, 'glycolysis'))
for enzyme in pentosephosphate_enzymes:
    c.execute("INSERT INTO enzyme_to_pathway (enzyme_name, pathway_name) VALUES (?,?);", (enzyme, 'pentophosphate'))
for enzyme in citrate_enzymes:
    c.execute("INSERT INTO enzyme_to_pathway (enzyme_name, pathway_name) VALUES (?,?);", (enzyme, 'citrate'))
        
c.execute("SELECT * FROM enzyme_to_pathway")
conn.commit()
for elem in c.fetchall():
    print(elem)
        

('phosphoglucomutase', 'glycolysis')
('glucose-6-phosphate isomerase', 'glycolysis')
('aldose 1-epimerase', 'glycolysis')
('6-phosphofructokinase', 'glycolysis')
('enolase', 'pentophosphate')
('phosphoglycerate kinase', 'pentophosphate')
('glyceraldehyde 3-phosphate dehydrogenase', 'pentophosphate')
('triosephosphate isomerase', 'pentophosphate')
('dihydrolipoamide dehydrogenase', 'citrate')
('alcohol dehydrogenase', 'citrate')
('acetyl-CoA synthetase', 'citrate')
('pyruvate kinase', 'citrate')


In [19]:
# GENE TO ENZYME TABLE (if my gene table were to be complete, this is the code that would output the right table)
c.execute("DROP TABLE gene_to_enzyme;")
c.execute("CREATE TABLE gene_to_enzyme (gene_name TEXT, enzyme_name TEXT);")
for index, row in df3.iterrows():
    c.execute("INSERT INTO gene_to_enzyme (gene_name, enzyme_name) VALUES (?, ?);",
             (row['gene_name'], row['product']))
c.execute("SELECT * FROM gene_to_enzyme;")
conn.commit()
for elem in c.fetchall():
    print(elem)

('HUMPGM1A', 'phosphoglucomutase')
('KY379509', 'glucose-6-phosphate isomerase')
('BC019263', 'aldose 1-epimerase')
('HUMPFK6PHO', '6-phosphofructokinase')
('X66610', 'enolase')
('AH002938', 'phosphoglycerate kinase')
('X01677', 'glyceraldehyde 3-phosphate dehydrogenase')
('X69723', 'triosephosphate isomerase')
('AH003583', 'dihydrolipoamide dehydrogenase')
('HUMALDE', 'alcohol dehydrogenase')
('AF263614', 'acetyl-CoA synthetase')
('HUMPKLR', 'pyruvate kinase')
('XM_023311391', 'phosphoglucomutase')
('XM_018103855', 'glucose-6-phosphate isomerase')
('XM_030714598', 'aldose 1-epimerase')
('AE013599', '6-phosphofructokinase')
('XM_023310563', 'enolase')
('XM_018002801', 'phosphoglycerate kinase')
('Z14144', 'glyceraldehyde 3-phosphate dehydrogenase')
('XM_030703036', 'triosephosphate isomerase')
('FJ907946', 'dihydrolipoamide dehydrogenase')
('DROADHDUPB', 'alcohol dehydrogenase')
('Z46786', 'acetyl-CoA synthetase')
('BK008741', 'pyruvate kinase')
('pgm', 'phosphoglucomutase')
('pgi', 'g